# Using DwC-A_dotnet.Interactive

This notebook describes how to use DwC-A_dotnet and DwC-A_dotnet.Interactive to interactively work with Darwin Core Archive files.

Information on the dotnet libraries used here may be found at 

|Library|Link|
|---|---|
|DwC-A_dotnet|https://github.com/pjoiner/DwC-A_dotnet|
|DwC-A_dotnet.Interactive|https://github.com/pjoiner/DwC-A_dotnet.Interactive|

Information on Darwin Core Archives may be found [here](https://dwc.tdwg.org/).

## Installation

Use the #r magic command to install the libraries from NuGet.

In [ ]:
#r "nuget:DwC-A_dotnet,0.5.1"
#r "nuget:DwC-A_dotnet.Interactive,0.1.4-Pre"


## Open An Archive
Use the `ArchiveReader` class to open the archive and provide the path to your archive.  It is recommended that the archive be unzipped to a directory first to reduce the overhead of creating a temporary folder to unzip the archive.  If you use the zip file remember to dispose of the temporary working directory at the end of you session by calling `archive.Dispose();`

In [1]:
using DwC_A;
using System.IO.Compression;
using System.IO;

Directory.Delete("./data/dwca-rooftop-v1.4", true);
ZipFile.ExtractToDirectory("./data/dwca-rooftop-v1.4.zip", "./data/dwca-rooftop-v1.4");
var archive = new ArchiveReader(@"./data/dwca-rooftop-v1.4");

## Archive MetaData
The interactive extensions library (`DwC-A_dotnet.Interactive`) registers kernel extensions to display various archive metadata by simply entering MetaData at the end of a cell.  The same can be done for an `IFileReader` instance to get a list of the term metadata.

In [ ]:
archive.MetaData

In [ ]:
archive.CoreFile

In [ ]:
archive.Extensions.GetFileReaderByFileName("occurrence.txt")

## Query Data Using LinQ

The following cell uses LinQ to gather a list of total individual counts of each genus for a specific sampling event.  Change the number in the `.Skip(1)` line to see totals calculated for other events. 

In [ ]:
using DwC_A.Terms;

//Retrieve the eventID from the event data file
var eventID = archive.CoreFile.DataRows
    .Skip(5)  //Change this number and run the cell again and to see the chart for a new eventID
    .Take(1)
    .First()[Terms.eventID];

//Get an IFileReader for the occurrence data file
var occurrences = archive.Extensions.GetFileReaderByFileName("occurrence.txt");

var data = occurrences.DataRows
    .Where(n => n[Terms.eventID] == eventID)
    .GroupBy(n => n[Terms.genus])
    .Select(g => new{
        Genus = g.Key,
        Count = g.Sum(c => int.Parse(c[Terms.individualCount])) 
    }); 

data